# Part 1 V2
This script is meant to be an update to the original Part 1, simplifying the functions. Some changes to original part 1 code will be needed. 

In [1]:
import pandas as pd
import numpy as np
import os

# Download All of the Data

In [23]:
# Information
DATASOURCE_IDS = ['35', '38', '41', '42']
j_drive_output_folder_path = 'J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_13_outputs/'
geography_levels = ['mgra_id', 'mgra', 'cpa', 'LUZ', 'jurisdiction', 'region']

In [4]:
# download all of the data 
all_data = {}
for ds_id in DATASOURCE_IDS:
    temp_dict = {}
    for file in os.listdir(j_drive_output_folder_path):
        if (ds_id in file):
            temp_dict[file] = pd.read_csv(j_drive_output_folder_path + file)
    all_data[ds_id] = temp_dict

# Both Files

In [8]:
def create_both_file(DS1, DS2, level, send_to_j_drive):
    df1 = all_data[DS1][f"forecast_{DS1}_{level}_ind.csv"]
    df2 = all_data[DS2][f"forecast_{DS2}_{level}_ind.csv"]

    # First check to see if shapes are identical
    if df1.shape != df2.shape:
        print("Shapes are not identical. DF1 is shape {df1.shape} and DF2 is shape {df2.shape}")
    elif len(set(df1.columns).difference(set(df2.columns))) != 0:
        print(f"There is a difference in columns. The columns that exist in DF1 but not in DF2 are {[x for x in df1.columns if x not in df2.columns]}. The columns that exist in DF2 but not in DF1 are {[x for x in df2.columns if x not in df1.columns]}")
    elif len(set(df1['yr_id']).difference(set(df2['yr_id']))) != 0:
        print(f"There is a difference in the years of these dataframes. The years that exist in DF1 but not in DF2 are {[x for x in list(set(df1['yr_id'])) if x not in list(set(df2['yr_id']))]}. The columns that exist in DF2 but not in DF1 are {[x for x in list(set(df2['yr_id'])) if x not in list(set(df1['yr_id']))]}")
    
    output = df1.merge(df2, on=[f'{level}','yr_id'], how='left', suffixes=[f'_DS{DS1}', f'_DS{DS2}'])
    output = output.groupby([f'{level}', 'yr_id']).sum()

    if send_to_j_drive == 'Y':
        output.to_csv('J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_13_outputs/' + f"{level}_both_DS{DS1}_DS{DS2}_QA.csv", index=True) 

    return output

In [56]:
both_file = create_both_file(DS1='35', DS2='41', level='cpa', send_to_j_drive='N')

In [57]:
both_file

10 to 14_DS35  15 to 17_DS35  18 and 19_DS35  \
cpa             yr_id                                                 
*Not in a CPA*  2016           94991          59660           38125   
                2018           92865          52913           42532   
                2020           97049          55070           42822   
                2025          102818          60418           47423   
                2030           91085          60405           51530   
...                              ...            ...             ...   
Via De La Valle 2030              30             24              24   
                2035              59             20              13   
                2040              45             31              26   
                2045              46             26              18   
                2050              43             26              16   

                       20 to 24_DS35  25 to 29_DS35  30 to 34_DS35  \
cpa             yr_id                                                
*Not in a CPA*  2016          107483          91639          92537   
                2018          118486         102526         102551   
                2020          116109         103687         100524   
                2025          121899          99341         100301   
                2030          131441         106085          95628   
...                              ...            ...            ...   
Via De La Valle 2030              53             35             32   
                2035              55             63             37   
                2040              55             52             46   
                2045              55             61             42   
                2050              53             54             52   

                       35 to 39_DS35  40 to 44_DS35  45 to 49_DS35  \
cpa             yr_id                                                
*Not in a CPA*  2016           89689          84417          89135   
                2018           94974          79920          83275   
                2020           99223          83735          77568   
                2025           95681          96031          79795   
                2030           93666          90869          90592   
...                              ...            ...            ...   
Via De La Valle 2030              29             29             35   
                2035              24             24             23   
                2040              43             32             27   
                2045              41             27             32   
                2050              38             42             26   

                       5 to 9_DS35  ...  Retail Trade_DS41  \
cpa             yr_id               ...                      
*Not in a CPA*  2016        102048  ...              79267   
                2018        100660  ...              81601   
                2020        101205  ...              82105   
                2025         91385  ...              84639   
                2030         86670  ...              90484   
...                            ...  ...                ...   
Via De La Valle 2030            57  ...                  0   
                2035            42  ...                  0   
                2040            35  ...                  0   
                2045            44  ...                  0   
                2050            33  ...                  0   

                       Self-Employed_DS41  \
cpa             yr_id                       
*Not in a CPA*  2016                    0   
                2018                 7009   
                2020                 7123   
                2025                 7415   
                2030                 7647   
...                                   ...   
Via De La Valle 2030                    2   
                2035                    2   
                2040

# Diff Files

In [64]:
def create_diff_file(DS1, DS2, level, send_to_j_drive):
    df1 = all_data[DS1][f"forecast_{DS1}_{level}_ind.csv"]
    df2 = all_data[DS2][f"forecast_{DS2}_{level}_ind.csv"]

    # First check to see if shapes are identical
    if df1.shape != df2.shape:
        print("Shapes are not identical. DF1 is shape {df1.shape} and DF2 is shape {df2.shape}")

    # Check if different columns are present amongst datasets, if so, manipulate so that only the shared columns are present
    if len(set(df1.columns).difference(set(df2.columns))) != 0:
        print(f"There is a difference in columns. The columns that exist in DF1 but not in DF2 are {[x for x in df1.columns if x not in df2.columns]}. The columns that exist in DF2 but not in DF1 are {[x for x in df2.columns if x not in df1.columns]}")
        # Ensure each data frame has the same columns
        df1 = df1[[x for x in df1.columns if x in df2.columns]]
        df2 = df2[[x for x in df1.columns if x in df2.columns]]

    # Check if different years are present amongst datasets, if so, manipulate so that only the shared years are compared
    if len(set(df1['yr_id']).difference(set(df2['yr_id']))) != 0:
        print(f"There is a difference in the years of these dataframes. The years that exist in DF1 but not in DF2 are {[x for x in list(set(df1['yr_id'])) if x not in list(set(df2['yr_id']))]}. The columns that exist in DF2 but not in DF1 are {[x for x in list(set(df2['yr_id'])) if x not in list(set(df1['yr_id']))]}")
        shared_years = [x for x in list(set(df1['yr_id'])) if x in list(set(df2['yr_id']))]
        df1 = df1[df1['yr_id'].isin(shared_years)]
        df2 = df2[df2['yr_id'].isin(shared_years)]
    
    df1 = df1.groupby([f'{level}', 'yr_id']).sum()
    df2 = df2.groupby([f'{level}', 'yr_id']).sum()

    diff_df = df2 - df1


    if send_to_j_drive == 'Y':
        diff_df.to_csv('J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_13_outputs/' + f"{level}_diff_DS{DS2}_minus_DS{DS1}_QA.csv", index=True) 

    return diff_df

In [67]:
result = create_diff_file(DS1='35', DS2='41', level='cpa', send_to_j_drive='N')

In [68]:
result

10 to 14  15 to 17  18 and 19  20 to 24  25 to 29  \
cpa             yr_id                                                      
*Not in a CPA*  2016          0         0          0         0         0   
                2018        198      -260        167        53       230   
                2020       -169      -139       -154        43       252   
                2025       -524        36        -77       479       797   
                2030       -114       116       -192      -339      -263   
...                         ...       ...        ...       ...       ...   
Via De La Valle 2030         10         0         -6        -5         2   
                2035        -23         9         -5        -7       -14   
                2040        -14         6         -9        -9         0   
                2045        -11         1          7         2        -9   
                2050        -13        -4          2         0         7   

                       30 to 34  35 to 39  40 to 44  45 to 49  5 to 9  ...  \
cpa             yr_id                                                  ...   
*Not in a CPA*  2016          0         0         0         0       0  ...   
                2018        132       105      -105        71     334  ...   
                2020       -170      -210       118       161     215  ...   
                2025        220      -285        82      -226    -186  ...   
                2030        167       630       272       218    -365  ...   
...                         ...       ...       ...       ...     ...  ...   
Via De La Valle 2030         15         9        -7        -9     -26  ...   
                2035        -11        12        15         7      -4  ...   
                2040          8       -12         7        16       2  ...   
                2045         11         7         1         2       1  ...   
                2050         -4         7        -5         6       0  ...   

                       Retail Trade  Self-Employed  \
cpa             yr_id                                
*Not in a CPA*  2016           -831              0   
                2018           -923              3   
                2020           -852              6   
                2025           -870             28   
                2030           -857             -5   
...                             ...            ...   
Via De La Valle 2030              0              0   
                2035              0              0   
                2040              0              0   
                2045              0             -1   
                2050              0             -1   

                       Transporation, Warehousing, and Utilities  \
cpa             yr_id                                              
*Not in a CPA*  2016                                        -160   
                2018                                        -155   
                2020                                        -156   
                2025                                        -181   
                2030                                        -152   
...                                                          ...   
Via De La Valle 2030                                           0   
                2035                                           0   
                2040                                           0   
                2045                                           0   
                2050                                           0   

                       Wholesale Trade  Group Quarters - College  \
cpa             yr_id                                              
*Not in a CPA*  2016                32                         0   
                2018                -1                         0   
                2020                -1                         0   
                2025                 4                         0   
                2030   